In [6]:
import numpy as np
import pandas as pd

#X = onehotencoder.fit_transform(imagesArray.reshape(-1,1)).toarray()
#y = dataSet["objeto"]
from keras.utils.np_utils import to_categorical
ames = pd.read_csv('ames.csv', nrows= 1e6)


In [8]:
np.random.seed(42)
datos.describe()
y = datos["precio_venta"]
variables_independientes = ames.drop("precio_venta", axis=1).columns
datos_numericos = ames[variables_independientes].select_dtypes([int, float])
col_no_numericas = ames[variables_independientes].select_dtypes([object]).columns
col_numericas = datos_numericos.columns

dict_var_ordinales = {
    "calidad_cocinas": ["Po", "Fa", "TA", "Gd", "Ex"],
    "funcionalidad":["Sal", "Sev", "Maj2", "Maj1", "Min2", "Min1","Typ"],
    "calidad_chimeneas":["NA","Po","Fa","TA","Gd","Ex"],
    "acabado_garaje":["NA","Unf","RFn","Fin"],
    "calidad_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_garaje":["NA","Po","Fa","TA","Gd","Ex"],
    "acceso_garaje_pavimentado":["N", "P", "Y"],
    "calidad_piscina":["NA","Fa","TA","Gd","Ex"],
    "calidad_valla":["NA","MnWw","GdWo","MnPrv","GdPrv"],
    "forma_parcela":["IR3", "IR2", "IR1","Reg"],
    "tipo_instalaciones":["ELO","NoSeWa","NoSewr","AllPub"],
    "pendiente_parcela":["Sev", "Mod", "Gtl"],
    "calidad_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "condicion_material_exterior":["Po","Fa","TA","Gd","Ex"],
    "altura_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "condicion_sotano":["NA","Po","Fa","TA","Gd","Ex"],
    "sotano_exterior":["NA","No","Mn","Av","Gd"],
    "calidad_sotano_habitable1":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "calidad_sotano_habitable2":["NA", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
     "calidad_calefaccion":["Po","Fa","TA","Gd","Ex"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = ames[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = ames[col_categoricas]



In [13]:
from sklearn.preprocessing import  Normalizer
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
datos_imputer = imp_mean.fit_transform(datos_numericos);

transformer = Normalizer().fit(datos_imputer)  # fit does nothing.
dator_normalizados = transformer.transform(datos_imputer)
datos_numericos_imputados_normalizados = pd.DataFrame(
    dator_normalizados,
    columns=datos_numericos.columns
)

In [14]:
datos_categoricos_dummy = pd.get_dummies(datos_categoricos, drop_first=True)

In [15]:
for columna_ordinal, valores in dict_var_ordinales.items():
    datos_ordinales[columna_ordinal] = (
    datos_ordinales[columna_ordinal]
    .astype("category")
    .cat.set_categories(valores)
    .cat.codes
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:

ames_procesado = pd.concat([
    datos_numericos_imputados_normalizados,
    datos_categoricos_dummy,
    datos_ordinales
], axis=1)
variable_objetivo = ["precio_venta"]
ames_X = ames_procesado
ames_y = ames[variable_objetivo]

from sklearn.feature_selection import SelectKBest, f_regression

selector_kbest10 = SelectKBest(f_regression, k=10)
ames_X_kbest10 = selector_kbest10.fit_transform(ames_X, ames_y)

ames_X_kbest10.shape

columnas_seleccion_kbest10 = ames_X.loc[:,selector_kbest10.get_support()].columns
columnas_seleccion_kbest10

datos_modelo = pd.DataFrame(
    ames_X_kbest10,
    columns=columnas_seleccion_kbest10
)



      area_sotano_total  ano_construccion_garaje  area_garage  \
0              0.443802                 0.805418     0.216970   
1              0.376930                 0.838049     0.311971   
2              0.512203                 0.754622     0.120246   
3              0.641088                 0.597944     0.158601   
4              0.387629                 0.834153     0.201333   
...                 ...                      ...          ...   
2925           0.409678                 0.810370     0.240170   
2926           0.379973                 0.872091     0.212855   
2927           0.400084                 0.868164     0.000000   
2928           0.517262                 0.735488     0.155663   
2929           0.405532                 0.811471     0.264655   

      tipo_andamios_PConc  calidad_cocinas  calidad_chimeneas  acabado_garaje  \
0                     0.0              2.0                4.0             3.0   
1                     0.0              2.0               

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test ,y_train,  y_test = train_test_split(datos_modelo, y)
modelo = LinearRegression()
modelo.fit(X=X_train,
           y=y_train)

y_pred = modelo.predict(X_test)

#print(X_train, X_test)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print(mse)

1966336206.4822652


In [4]:
from sklearn.externals import joblib

joblib.dump(modelo, 'modelo_ames.pkl') 
import json
with open('columnas_names.json', 'w') as fname:
    x_columnas = X.columns.tolist()
    json.dump(x_columnas, fname)
    
x_dtypes = X.dtypes
x_dtypes = {col: X[col].dtype for col in X.columns}
joblib.dump(x_dtypes, 'dtypes_ames.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['dtypes_ames.pkl']

,area_parcela,area_piscina,n_chimeneas,n_cocinas,n_dormitorios_sobre_suelo,area_habitable_sobre_suelo,calidad_general
0,31770,0,2,1,3,1656,6
1,11622,0,0,1,2,896,5
2,14267,0,0,1,3,1329,6
3,11160,0,2,1,3,2110,7
4,13830,0,1,1,3,1629,5
...,...,...,...,...,...,...,...
2925,7937,0,0,1,3,1003,6
2926,8885,0,0,1,2,902,5
2927,10441,0,0,1,3,970,5
2928,10010,0,1,1,2,1389,5
